Nhóm 9:\
23122018 Lại Nguyễn Hồng Thanh\
23122019 Phan Huỳnh Châu Thịnh\
23122029 Nguyễn Trọng Hòa\
23122047 Nguyễn Xuân Quang

# Cài đặt các thư viện

In [1]:
import pandas as pd
import numpy as np

# Tải tập dữ liệu

## Hàm đọc dữ liệu từ link google drive

In [2]:
def download_drive_csv(file_id, filename='data.csv'):
    url = f"https://drive.google.com/uc?id={file_id}"

    try:
        df = pd.read_csv(url)
        print(f"Đã tải và đọc thành công file CSV từ Google Drive. Kích thước: {df.shape}")
        return df
    except Exception as e:
        print("Lỗi khi tải file:", e)
        return None

## Đọc tập train: [train.csv](https://drive.google.com/file/d/1sq2Cia5Nc1dwWubC9NAGz7jzaRK5nZ9-/view?usp=drive_link)

In [3]:
#Đọc dữ liệu
file_id_train = "1sq2Cia5Nc1dwWubC9NAGz7jzaRK5nZ9-"
train = download_drive_csv(file_id_train)

#In ra 5 dòng đầu tiên của tập train
train.head()

Đã tải và đọc thành công file CSV từ Google Drive. Kích thước: (27284, 6)


,Unnamed: 0,Message ID,Subject,Message,Spam/Ham,split
0,0,0,christmas tree farm pictures,NaN,ham,0.038415
1,1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,0.696509
2,2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,0.587792
3,3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,-0.055438
4,5,5,mcmullen gas for 11 / 99,"jackie ,\nsince the inlet to 3 river plant is ...",ham,-0.419658


## Đọc tập val: [val.csv](https://drive.google.com/file/d/1gF8xTQEbUhtIpezYnrTE3wuonIhfx7NC/view?usp=drive_link)

In [4]:
#Đọc dữ liệu
file_id_val = "1gF8xTQEbUhtIpezYnrTE3wuonIhfx7NC"
val = download_drive_csv(file_id_val)
#In ra 5 dòng đầu tiên của tập val
val.head()

Đã tải và đọc thành công file CSV từ Google Drive. Kích thước: (3084, 6)


,Unnamed: 0,Message ID,Subject,Message,Spam/Ham,split
0,23,23,miscellaneous,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,-0.351998
1,24,24,re : purge of old contract _ event _ status,fyi - what do you all think ?\n- - - - - - - -...,ham,0.257704
2,32,32,valero 8018 and 1394,it is my understanding the outages valero incu...,ham,0.091200
3,37,37,01 / 00 natural gas nomination,enron methanol company nominates the following...,ham,-1.745133
4,43,43,re : misc . questions,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,-1.911987


# Tiền xử lý dữ liệu

## Xử lý dữ liệu trùng lặp

### *Kiểm tra dữ liệu trùng lặp*

In [5]:
#Tập train
num_dupl_train = train.duplicated().sum()
print("Số lượng dữ liệu trùng lặp trong tập train:", num_dupl_train)

#Tập val
num_dupl_val = val.duplicated().sum()
print("Số lượng dữ liệu trùng lặp trong tập val:", num_dupl_val)

Số lượng dữ liệu trùng lặp trong tập train: 0
Số lượng dữ liệu trùng lặp trong tập val: 0


=> Không có dữ liệu trùng lặp

## Xử lý dữ liệu thiếu

###*Kiểm tra dữ liệu thiếu*

In [6]:
#Tập train
miss_vals_train = train.isnull().sum()
miss_vals_train = miss_vals_train[miss_vals_train > 0]

#Tập val
miss_vals_val = val.isnull().sum()
miss_vals_val = miss_vals_val[miss_vals_val > 0]

print("Số lượng dữ liệu thiếu: ")
print("Tập train: \n", miss_vals_train)
print("Tập val: \n", miss_vals_val)

Số lượng dữ liệu thiếu: 
Tập train: 
 Subject    229
Message    352
dtype: int64
Tập val: 
 Subject    29
Message    35
dtype: int64


### *Xử lý dữ liệu thiếu*
Cách 1: Xóa những dòng dữ liệu thiếu\
Cách 2: Thay bằng fillna(' ')

#### **Xóa những dòng thiếu**

In [7]:
"""#Tập train
train = train.dropna(subset = ['Subject', 'Message']).reset_index(drop = True)

#Tập val
val = val.dropna(subset = ['Subject', 'Message']).reset_index(drop = True)

print("Kích thước sau khi xóa:")
print("Train: ", train.shape)
print("Val:", val.shape)"""

'#Tập train\ntrain = train.dropna(subset = [\'Subject\', \'Message\']).reset_index(drop = True)\n\n#Tập val\nval = val.dropna(subset = [\'Subject\', \'Message\']).reset_index(drop = True)\n\nprint("Kích thước sau khi xóa:")\nprint("Train: ", train.shape)\nprint("Val:", val.shape)'

#### **Xử lý bằng fillna**

In [8]:
# Tập train
train = train.fillna('')

# Tập val
val = val.fillna('')

## Trộn lại dữ liệu
- Tránh dữ liệu bị lệch phân phối spam/ham theo thứ tự: các thư spam nằm cùng một khu, các thư ham nằm cùng một khu.
- Giúp các nhãn spam/ham được phân bố đều khắp tập dữ liệu.

In [9]:
#Tập train
train = train.sample(frac = 1, random_state = 42).reset_index(drop = True)

## Gộp dữ liệu
Gộp **Subject** và **Message** thành cột **text**: giúp mô hình học được xác suất từ trong cả subject và message

In [10]:
#Tập train
train['text'] = train['Subject'] + ' ' + train['Message']

#Tập val
val['text'] = val['Subject'] + ' ' + val['Message']

## Kiểm tra tỷ lệ spam/ham

In [11]:
print("Train:")
print(train['Spam/Ham'].value_counts(normalize=True))
print("Val:")
print(val['Spam/Ham'].value_counts(normalize=True))

Train:
Spam/Ham
spam    0.507917
ham     0.492083
Name: proportion, dtype: float64
Val:
Spam/Ham
spam    0.506809
ham     0.493191
Name: proportion, dtype: float64


=> Tỷ lệ spam/ham tương đối đồng đều.

## Mã hóa dữ liệu X, Y

### *Dùng Bag-of-words*

In [12]:
# Mã hóa BoW
def encode_bow_sparse(texts, vocab):
    from scipy.sparse import lil_matrix
    matrix = lil_matrix((len(texts), len(vocab)))
    for i, text in enumerate(texts):
        for word in text.split():
            if word in vocab:
                matrix[i, vocab[word]] += 1
    return matrix.tocsr()

# Xây dựng từ vựng
vocab = {}
idx = 0
for text in train['text']:
    for word in text.split():
        if word not in vocab:
            vocab[word] = idx
            idx += 1


X_train = encode_bow_sparse(train['text'], vocab)
X_val = encode_bow_sparse(val['text'], vocab)

Y_train = train['Spam/Ham'].map({'ham': 0, 'spam': 1}).to_numpy()
Y_val = val['Spam/Ham'].map({'ham': 0, 'spam': 1}).to_numpy()

# Mô hình

## Mô hình: Naive Bayes Classifer


In [13]:
class MultinomialNB_self:
    def __init__(self, alpha=1.0, threshold=0.5):
        self.alpha = alpha
        self.threshold = threshold
        self.class_prior_ = None
        self.feature_prob_ = None
        self.classes_ = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        n_features = X.shape[1]

        count_samples = np.array([np.sum(y == c) for c in self.classes_])
        self.class_prior_ = count_samples / y.shape[0]

        feature_count = np.zeros((n_classes, n_features), dtype=np.float64)
        for idx, c in enumerate(self.classes_):
            feature_count[idx, :] = X[y == c].sum(axis=0)

        N_c = feature_count.sum(axis=1)

        self.feature_prob_ = (feature_count + self.alpha) / (N_c[:, np.newaxis] + self.alpha * n_features)

    def predict(self, X):
        log_feature_prob = np.log(self.feature_prob_)
        log_class_prior = np.log(self.class_prior_)

        log_likelihood = X @ log_feature_prob.T
        log_posterior = log_likelihood + log_class_prior

        max_log = np.max(log_posterior, axis=1, keepdims=True)
        exp_log = np.exp(log_posterior - max_log)
        posterior_prob = exp_log / np.sum(exp_log, axis=1, keepdims=True)

        spam_index = np.where(self.classes_ == 1)[0][0]
        preds = (posterior_prob[:, spam_index] > self.threshold).astype(int)
        return preds


In [14]:
model_self = MultinomialNB_self(alpha=1.0, threshold=0.5)
model_self.fit(X_train, Y_train)

train_pred = model_self.predict(X_train)
val_pred = model_self.predict(X_val)

## Đánh giá mô hình

In [15]:
def compute_metrics(y_true, y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)

  TP = np.sum((y_true == 1) & (y_pred == 1))
  TN = np.sum((y_true == 0) & (y_pred == 0))
  FP = np.sum((y_true == 0) & (y_pred == 1))
  FN = np.sum((y_true == 1) & (y_pred == 0))

  if len(y_true) > 0:
    accuracy = (TP + TN) / len(y_true)
  else:
    accuracy = 0

  if (TP + FN) > 0:
    recall = TP / (TP + FN)
  else:
    recall = 0

  if (TP + FP) > 0:
    precision = TP / (TP + FP)
  else:
    precision = 0

  if (precision + recall) > 0:
    f1 = 2 * precision * recall / (precision + recall)
  else:
    f1 = 0

  return accuracy, recall, precision, f1

In [16]:
print("Train:")
acc, rec, prec, f1 = compute_metrics(Y_train, train_pred)
print("Accuracy :", acc)
print("Recall   :", rec)
print("Precision:", prec)
print("F1-score :", f1)

print("Validation:")
acc, rec, prec, f1 = compute_metrics(Y_val, val_pred)
print("Accuracy :", acc)
print("Recall   :", rec)
print("Precision:", prec)
print("F1-score :", f1)

Train:
Accuracy : 0.9892977569271367
Recall   : 0.9903304950209265
Precision: 0.9886183547039331
F1-score : 0.9894736842105262
Validation:
Accuracy : 0.9850843060959793
Recall   : 0.9852847088931542
Precision: 0.9852847088931542
F1-score : 0.9852847088931542


# Thử nghiệm thực tế

## Chức năng 1

### **Xử lý như xử lý train**

In [17]:
def process_email(subject, message, model, vocab):
  #Không cần xử lý trùng lặp vì chỉ có một email
  #Xử lý nếu thiếu subject hay message
  if not subject:
    subject = ' '
  if not message:
    message = ' '

  #Gộp dữ liệu
  text = subject + ' ' + message

  #Mã hóa
  if vocab is not None:
    X = encode_bow_sparse([text], vocab)
  else:
    raise ValueError("Vocab must be provided")
  return X

### **Phân loại**

In [18]:
def predict_email(subject, message, model, vocab):
  X = process_email(subject, message, model, vocab)
  pred = model.predict(X)
  return pred

In [19]:
subject_input = input("Nhập vào tiêu đề email (Subject): ")
message_input = input("Nhập vào nội dung email (Message): ")

label = predict_email(subject_input, message_input, model_self, vocab = vocab)
if label == 1:
  print("Email là spam")
else:
  print("Email là ham")


Nhập vào tiêu đề email (Subject): hhh
Nhập vào nội dung email (Message): kk
Email là spam


## Chức năng 2
Đọc dữ liệu từ file test.csv (Vui lòng truyền vào **file_path**)

### **Xử lý dữ liệu như train**

In [20]:
def process_test(file_path, model, vocab):
  test = pd.read_csv(file_path)

  #Xử lý dữ liệu trùng lặp
  num_dupl = test.duplicated().sum()
  if (num_dupl > 0):
    test = test.drop_duplicates()

  #Xử lý dữ liệu thiếu
  miss_vals = test.isnull().sum()
  miss_vals = miss_vals[miss_vals > 0]

  if (miss_vals.shape[0] > 0):
    test = test.fillna(' ')

  #Gộp dữ liệu
  test['text'] = test['Subject'] + ' ' + test['Message']

  #Mã hóa
  if vocab is not None:
    X = encode_bow_sparse(test['text'], vocab)
  else:
    raise ValueError("Vocab must be provided")

  Y = test['Spam/Ham'].map({'ham': 0, 'spam': 1}).to_numpy()
  return X, Y

In [21]:
X, Y = process_test("test.csv", model_self, vocab)

### **Phân loại Email**

In [22]:
def predict_test(X, model, vocab):
  pred = model.predict(X)
  return pred

In [23]:
label = predict_test(X, model_self, vocab = vocab)

### **Đánh giá**

In [ ]:
acc, rec, prec, f1 = compute_metrics(Y, label)
print("Accuracy :", acc)
print("Recall   :", rec)
print("Precision:", prec)
print("F1-score :", f1)